# Schema Enforcement y Schema Evolution en Databricks

## Introducción

En el mundo de los datos, los esquemas definen la estructura de nuestras tablas: nombres de columnas, tipos de datos, restricciones, etc. Cuando trabajamos con pipelines de datos, especialmente en entornos donde los datos provienen de múltiples fuentes o evolucionan con el tiempo, necesitamos mecanismos para manejar cambios en la estructura de los datos.

Databricks proporciona dos conceptos fundamentales para manejar esquemas de manera robusta:

1. **Schema Enforcement**: Validación estricta de esquemas durante operaciones de escritura
2. **Schema Evolution**: Adaptación automática o controlada de esquemas cuando cambian los datos

Esta notebook explica ambos conceptos y muestra ejemplos prácticos aplicados a nuestras tablas de ingesta.

## Parte 1: Schema Enforcement

### ¿Qué es Schema Enforcement?

Schema Enforcement es una **validación estricta** que Databricks aplica automáticamente cuando escribimos datos en tablas Delta Lake. Esta validación asegura la calidad de los datos verificando que:

- Todas las columnas que se intentan insertar existen en la tabla destino
- Los tipos de datos de las columnas coinciden exactamente con el esquema definido
- No se permiten inserciones que violen la estructura esperada

### Comportamiento por defecto en Delta Lake

Por defecto, **todas las tablas Delta Lake tienen Schema Enforcement habilitado**. Esto significa que cualquier operación de escritura que no cumpla con el esquema exacto de la tabla fallará.

**Reglas de validación:**

1. **Columnas existentes**: Todas las columnas en los datos fuente deben existir en la tabla destino
2. **Tipos de datos**: Los tipos de datos deben coincidir exactamente
3. **Columnas nuevas**: No se permiten columnas adicionales (generan error)

### Ejemplo de Schema Enforcement

```sql
-- Supongamos que tenemos esta tabla:
CREATE TABLE mi_tabla (
  id INT,
  nombre STRING,
  fecha DATE
);

-- Esta inserción funciona (cumple con el esquema):
INSERT INTO mi_tabla VALUES (1, 'Juan', '2024-01-01');

-- Esta inserción FALLA (columna adicional):
INSERT INTO mi_tabla VALUES (1, 'Juan', '2024-01-01', 'extra'); -- ERROR

-- Esta inserción FALLA (tipo de dato incorrecto):
INSERT INTO mi_tabla VALUES ('1', 'Juan', '2024-01-01'); -- ERROR: id debe ser INT
```

### Consideraciones importantes

**Ventajas:**
- ✅ **Calidad de datos**: Previene inserciones incorrectas
- ✅ **Consistencia**: Mantiene la estructura esperada
- ✅ **Detección temprana**: Errores se detectan inmediatamente

**Desventajas:**
- ❌ **Rigidez**: No permite cambios naturales en los datos
- ❌ **Mantenimiento**: Requiere actualizaciones manuales del esquema
- ❌ **Flexibilidad limitada**: Problemas con datos semi-estructurados

### Schema Enforcement en streaming

Para streams continuos, Schema Enforcement es crítico porque:
- Los streams esperan un esquema consistente
- Cambios inesperados pueden romper el procesamiento
- La validación ocurre en cada micro-batch

## Parte 2: Schema Evolution

### ¿Qué es Schema Evolution?

Schema Evolution es la **capacidad de un sistema para adaptarse a cambios en la estructura de los datos** sin requerir intervención manual completa. En Databricks, esto permite:

- Agregar nuevas columnas automáticamente
- Cambiar tipos de datos de manera controlada
- Renombrar columnas sin perder datos
- Adaptarse a esquemas que evolucionan naturalmente

### Tipos de cambios en Schema Evolution

Los cambios más comunes que maneja Schema Evolution incluyen:

1. **Nuevas columnas**: Campos adicionales que aparecen en los datos
2. **Renombrado de columnas**: Cambios en nombres de campos
3. **Eliminación de columnas**: Campos que dejan de existir
4. **Cambio de tipos**: Modificaciones en tipos de datos (ampliación de tipos de datos)
5. **Cambios de tipos arbitrarios**: Transformaciones más complejas

### Type Widening (Ampliación de tipos de datos)

Type widening es una característica avanzada de Schema Evolution que permite **cambiar tipos de datos a tipos más amplios sin reescribir los archivos de datos subyacentes**. Esta funcionalidad está disponible en **Databricks Runtime 15.4 LTS y superior**.

#### ¿Qué permite type widening?

Type widening permite cambiar tipos de datos siguiendo reglas específicas de **ampliación** (de tipos más restrictivos a más amplios):

| Tipo fuente | Tipos más amplios soportados |
|-------------|-----------------------------|
| byte       | short, int, long, decimal, double |
| short      | int, long, decimal, double |
| int        | long, decimal, double |
| long       | decimal |
| float      | double |
| decimal    | decimal con mayor precisión y escala |
| date       | timestampNTZ |

**Importante**: Los cambios de tipos enteros (`byte`, `short`, `int`, `long`) a `decimal` o `double` deben hacerse **manualmente** para evitar promociones accidentales.

#### Ejemplo de type widening

```sql
-- Habilitar type widening en la tabla
ALTER TABLE mi_tabla SET TBLPROPERTIES ('delta.enableTypeWidening' = 'true');

-- Cambiar tipo de columna sin reescribir datos
ALTER TABLE mi_tabla ALTER COLUMN edad TYPE INT;  -- de SMALLINT a INT
ALTER TABLE mi_tabla ALTER COLUMN precio TYPE DECIMAL(10,2);  -- de DECIMAL(5,2) a DECIMAL(10,2)
```

#### Type widening con schema evolution automática

Type widening se puede combinar con schema evolution para **cambios automáticos de tipos** durante la ingesta:

```python
# Escritura con schema evolution y type widening
(df.write
    .option("mergeSchema", "true")
    .mode("append")
    .saveAsTable("mi_tabla")
)
```

**Condiciones para type widening automático:**
- Schema evolution habilitado (`mergeSchema=true`)
- Type widening habilitado en la tabla destino
- El tipo fuente es más amplio que el tipo destino
- El cambio está soportado por las reglas de type widening

### Arquitectura de Schema Evolution en Databricks

Databricks maneja Schema Evolution a través de **4 componentes independientes**:

1. **Conectores**: Componentes que ingestan datos externos
2. **Parsers de formato**: Funciones que decodifican formatos raw
3. **Engines**: Motores de procesamiento (Structured Streaming)
4. **Datasets**: Almacenes finales (Delta tables, views, etc.)

Cada componente maneja Schema Evolution de manera independiente, requiriendo configuración específica.

### Schema Evolution por componente

#### Auto Loader
- ✅ **Nuevas columnas**: Soportado (con restart)
- ✅ **Renombrado**: Soportado (tratado como nueva columna + NULL)
- ❌ **Tipos widening**: No soportado
- ❌ **Tipos arbitrarios**: No soportado

#### Delta Connector
- ✅ **Nuevas columnas**: Soportado
- ✅ **Renombrado**: Soportado con configuración específica
- ✅ **Tipos widening**: Soportado con configuración
- ✅ **Tipos arbitrarios**: Soportado (requiere rewrite)

#### Structured Streaming
- ✅ **Todos los cambios**: Soportados (requieren restart del stream)

#### Delta Tables
- ✅ **Nuevas columnas**: Auto-evolución con `mergeSchema`
- ✅ **Renombrado**: Con `ALTER TABLE` + column mapping
- ✅ **Tipos widening**: Con table property específica
- ✅ **Tipos arbitrarios**: Con `overwriteSchema`

### Consideraciones importantes

**Cuándo usar Schema Evolution:**
- ✅ Datos semi-estructurados que evolucionan naturalmente
- ✅ Pipelines que deben ser resilientes a cambios
- ✅ Equipos con ciclos de desarrollo rápidos

**Cuándo evitar Schema Evolution:**
- ❌ Esquemas estables con contratos estrictos
- ❌ Requisitos de calidad que demandan validación estricta
- ❌ Entornos regulados con cambios controlados

**Mejores prácticas:**
- Configurar evolución por operación, no globalmente
- Monitorear cambios de esquema
- Documentar evoluciones importantes
- Realizar pruebas exhaustivas antes de aplicar cambios en producción

## Parte 3: Ejemplo práctico - Schema Evolution en carga JDBC

Basado en el notebook `01. carga_full_jdbc.ipynb`, vamos a mostrar cómo aplicar Schema Evolution a las tablas cargadas desde PostgreSQL.

### Escenario

Supongamos que nuestra tabla `accounts` en PostgreSQL agrega una nueva columna `account_status` que indica si la cuenta está activa. Queremos que nuestra tabla Delta evolucione automáticamente para incluir esta columna.

### Código de ejemplo con Schema Evolution

In [0]:
# La configuración es igual que el notebook original
# Acá suponemos que en df_strings tenemos una nueva columna account_status que apareció en la tabla origen
# ESCRITURA CON SCHEMA EVOLUTION
# La diferencia clave: mergeSchema="true" permite agregar nuevas columnas
(df_strings.write
    .mode("overwrite")
    .option("replaceWhere", f"fecha_extraccion = '{today}'")
    .option("mergeSchema", "true")  # <-- HABILITA SCHEMA EVOLUTION
    .saveAsTable(target)
)

print("Tabla procesada con Schema Evolution habilitado")

### ¿Qué cambia con Schema Evolution?

**Sin Schema Evolution (por defecto):**
- Si `accounts` agrega `account_status`, la escritura falla
- Error: "Cannot add column account_status to table accounts"

**Con Schema Evolution (`mergeSchema=true`):**
- La columna `account_status` se agrega automáticamente
- Registros existentes tienen `NULL` para esa columna
- Nuevos registros incluyen los valores de `account_status`

### Configuración adicional para tipos de cambios avanzados

Si necesitamos manejar cambios más complejos, podemos configurar propiedades adicionales:

In [0]:
# Habilitar type widening para cambios de tipos
spark.sql(f"""
ALTER TABLE {target} SET TBLPROPERTIES (
    'delta.typeWidening.enabled' = 'true'
)
""")

# Habilitar column mapping para renombrado
spark.sql(f"""
ALTER TABLE {target} SET TBLPROPERTIES (
    'delta.columnMapping.mode' = 'name'
)
""")

## Parte 4: Schema Evolution en Auto Loader

### ¿Cómo funciona Schema Evolution en Auto Loader?

Auto Loader tiene un **sistema sofisticado de inferencia y evolución de esquemas** que permite manejar cambios en la estructura de datos de manera automática.

#### Inferencia inicial de esquema

Cuando Auto Loader procesa datos por primera vez:
- **Muestrea** los primeros 50 GB o 1000 archivos (lo que ocurra primero)
- **Infiera el esquema** basado en el formato del archivo:
  - **JSON/CSV/XML**: Todas las columnas se infieren como `STRING`
  - **Parquet/Avro**: Usa los tipos codificados en el esquema del archivo
- **Almacena** la información del esquema en el directorio `_schemas` dentro de `cloudFiles.schemaLocation`

#### Modos de evolución de esquema

Auto Loader soporta diferentes modos de evolución configurados con `cloudFiles.schemaEvolutionMode`:

| Modo | Comportamiento |
|------|----------------|
| `addNewColumns` (default) | **Stream falla**. Nuevas columnas se agregan al esquema. Tipos de columnas existentes no evolucionan. |
| `rescue` | **Esquema nunca evoluciona**. Nuevas columnas van a la columna de datos rescatados. |
| `failOnNewColumns` | **Stream falla**. No reinicia hasta actualizar el esquema manualmente. |
| `none` | **No evoluciona esquema**. Nuevas columnas se ignoran (a menos que se configure `rescuedDataColumn`). |

**Nota**: `addNewColumns` es el default cuando no se proporciona esquema, pero `none` es el default cuando se proporciona un esquema.

#### Proceso de evolución

1. **Detección**: Auto Loader detecta nuevas columnas en el micro-batch actual
2. **Inferencia**: Realiza nueva inferencia de esquema en el batch actual
3. **Merge**: Fusiona nuevas columnas al final del esquema existente
4. **Almacenamiento**: Actualiza el esquema en `cloudFiles.schemaLocation`
5. **Falla del stream**: El stream se detiene con `UnknownFieldException`
6. **Reinicio**: Al reiniciar, usa el esquema evolucionado

**Recomendación**: Configurar streams de Auto Loader con Lakeflow Jobs para reinicio automático tras cambios de esquema.

#### Columna de datos rescatados (_rescued_data)

Auto Loader automáticamente agrega una columna `_rescued_data` que contiene:
- **Columnas nuevas** no previstas en el esquema
- **Tipos de datos incompatibles**
- **Diferencias de case** en nombres de columnas
- **Ruta del archivo fuente** del registro

```python
# La columna contiene JSON con datos no parseados
# Ejemplo: {"new_column": "value", "source_file": "/path/to/file.csv"}
```

#### Schema Hints para sobrescribir inferencia

Puedes usar `schemaHints` para forzar tipos de datos específicos:

```python
(spark.readStream.format("cloudFiles")
    .option("cloudFiles.format", "csv")
    .option("cloudFiles.schemaLocation", checkpoint_path)
    .option("cloudFiles.schemaHints", "fecha DATE, precio DECIMAL(10,2), tags MAP<STRING,STRING>")
    .load(source_path)
    .writeStream...
)
```

**Sintaxis de schemaHints**:
- `columna TIPO` - Para tipos simples
- `columna.nested TIPO` - Para campos anidados
- `ARRAY<TIPO>` - Para arrays
- `MAP<KEY_TYPE,VALUE_TYPE>` - Para maps
- `STRUCT<campo1:TIPO1, campo2:TIPO2>` - Para structs

### Ejemplo práctico - Schema Evolution en Auto Loader

Basado en el notebook `02. carga_incremental_autoloader.ipynb`, vamos a mostrar cómo aplicar Schema Evolution a la tabla `account_premium_features` cargada con Auto Loader.

#### Escenario

Supongamos que los archivos CSV de `account_premium_features` comienzan a incluir una nueva columna `discount_applied` que indica si se aplicó un descuento. Queremos que nuestra tabla Delta evolucione automáticamente.

#### Código de ejemplo con Schema Evolution

In [0]:
# Configuración igual que el notebook original

# CARGA INCREMENTAL CON SCHEMA EVOLUTION EN AUTO LOADER
query = (spark.readStream
  .format("cloudFiles")
  
  # Configuración de Auto Loader con evolución de esquema
  .option("cloudFiles.format", "csv")
  .option("cloudFiles.schemaLocation", checkpoint_path)
  
  # MODO DE EVOLUCIÓN: addNewColumns (default) - permite nuevas columnas
  .option("cloudFiles.schemaEvolutionMode", "addNewColumns")
  
  # HABILITAR COLUMNA DE DATOS RESCATADOS para datos inesperados
  .option("rescuedDataColumn", "_rescued_data")
  
  # SCHEMA HINTS: forzar tipos específicos si es necesario
  # .option("cloudFiles.schemaHints", "purchase_date DATE, amount_paid DECIMAL(10,2)")
  
  .load(f"{VOLUMEN_LANDING}/{TABLA}/")
  
  # ESCRITURA CON SCHEMA EVOLUTION PARA DELTA TABLE
  .writeStream
  .option("checkpointLocation", checkpoint_path)
  .trigger(availableNow=True)
  
  # HABILITAR SCHEMA EVOLUTION PARA LA TABLA DELTA
  .option("mergeSchema", "true")  # <-- Permite nuevas columnas en tabla Delta
  
  .toTable(TARGET_TABLE)
)

print("Carga incremental completada con Schema Evolution")

### ¿Qué habilita Schema Evolution en Auto Loader?

**Configuración en Auto Loader (`cloudFiles.schemaEvolutionMode`):**
- **`addNewColumns`**: Detecta nuevas columnas y las agrega al esquema (stream falla inicialmente)
- **`rescue`**: Envía datos inesperados a columna `_rescued_data` sin fallar
- **`failOnNewColumns`**: Falla completamente ante nuevas columnas
- **`none`**: Ignora nuevas columnas completamente

**Configuración en Delta Table (`mergeSchema`):**
- Nuevas columnas se agregan automáticamente a la tabla Delta
- Registros existentes tienen `NULL` en columnas nuevas
- Compatible con type widening si está habilitado

**Flujo completo de evolución:**
1. **Auto Loader detecta** nuevas columnas en archivos CSV
2. **Stream falla** con `UnknownFieldException` (modo `addNewColumns`)
3. **Esquema se actualiza** en `cloudFiles.schemaLocation`
4. **Al reiniciar**, el stream procesa con esquema evolucionado
5. **Tabla Delta** agrega columnas automáticamente con `mergeSchema=true`

**Comportamiento específico:**
- Si un archivo CSV incluye `discount_applied`, Auto Loader lo detecta
- Datos van inicialmente a `_rescued_data` o stream falla
- Después del reinicio, `discount_applied` se agrega como columna regular
- Registros anteriores tienen `NULL` en la nueva columna

### Manejo de la columna _rescued_data

La columna `_rescued_data` contiene datos que no pudieron ser parseados según el esquema actual:

```sql
-- Ver datos rescatados
SELECT account_id, _rescued_data
FROM account_premium_features
WHERE _rescued_data IS NOT NULL
LIMIT 10;
```

**Contenido típico de _rescued_data:**
- Nuevas columnas detectadas
- Tipos de datos incompatibles
- Diferencias de capitalización en nombres
- Ruta del archivo fuente

## Conclusiones

Schema Enforcement y Schema Evolution son mecanismos complementarios para manejar la evolución de datos en entornos de producción:

- **Schema Enforcement**: Proporciona validación estricta y consistencia
- **Schema Evolution**: Permite adaptabilidad y resiliencia a cambios

**Recomendaciones:**
- Usar Schema Enforcement por defecto para calidad de datos
- Habilitar Schema Evolution selectivamente donde sea necesario
- Monitorear cambios de esquema en producción
- Documentar evoluciones importantes
- Probar cambios exhaustivamente antes de aplicar en producción